In [169]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf



In [170]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

n = 100
d = 10
noise_factor = 0.01

# Create (noisy) testing data for binary classification.
X, y = make_classification(
    n_samples=n, 
    n_features=d,
    n_informative=d,
    n_redundant=0, 
    n_classes=2,
    class_sep=-1,
    flip_y=noise_factor
)

# We will work with label values -1, +1 and not 0, +1 (convert)
y[y == 0] = -1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


In [171]:
# Convert the data to TensorFlow tensors
X_train_tensor = tf.constant(X_train, dtype=tf.float32)
y_train_tensor = tf.constant(y_train, dtype=tf.float32)
X_test_tensor = tf.constant(X_test, dtype=tf.float32)
y_test_tensor = tf.constant(y_test, dtype=tf.float32)

In [172]:
# Combine the X and y tensors into a single dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tensor, y_test_tensor))

In [173]:
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 5
SHUFFLE_BUFFER = 1
PREFETCH_BUFFER = 2

def preprocess(dataset):
    return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER, seed=1).batch(BATCH_SIZE).prefetch(PREFETCH_BUFFER)

In [174]:

# Preprocess the training dataset
preprocessed_train_dataset = preprocess(train_dataset)

# Preprocess the testing dataset
preprocessed_test_dataset = preprocess(test_dataset)

In [201]:
def t_pa1(x_batch, loss, C):
        return tf.minimum(C, tf.divide(loss, tf.reduce_sum(tf.square(x_batch), axis=1)))

def _train_on_batch(weights, batch, C, t_fn):
        x_batch, y_batch = batch
        
        # predict y
        y_predict_batch = tf.sign(tf.reduce_sum(tf.multiply(weights, x_batch), axis=0))
        # suffer loss
        loss = tf.maximum(0, tf.subtract(1, tf.multiply(y_predict_batch, tf.reduce_sum(tf.multiply(weights, x_batch), axis=0))))
        # update 1. Set
        t = t_fn(loss, x_batch, C)
        # update 1. Update
        weights = tf.reduce_mean(tf.multiply(tf.expand_dims(tf.multiply(t, y_batch), axis=1), x_batch), axis=0)
        
        return weights

In [161]:
model = tf.zeros(shape=(d,), dtype=tf.float32)

batch = next(iter(preprocessed_test_dataset))

_train_on_batch(model, batch, 0.01, t_pa1)

InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,10], In[1]: [5,10] [Op:MatMul]

In [204]:
# predict y
y_predict_batch = tf.sign(tf.reduce_sum(tf.multiply(model, x_batch), axis=0))
# suffer loss
loss = tf.maximum(0, tf.subtract(1, tf.multiply(y_predict_batch, tf.reduce_sum(tf.multiply(model, x_batch), axis=0))))

# update 1. Set
t = tf.minimum(0.01, tf.divide(loss, tf.reduce_sum(tf.square(x_batch), axis=1)))




# update 1. Update
model = tf.reduce_mean(tf.multiply(tf.expand_dims(tf.multiply(t, y_batch), axis=1), x_batch), axis=0)

InvalidArgumentError: {{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [10] vs. [5] [Op:RealDiv]

In [208]:
tf.reduce_sum(tf.square(x_batch), axis=1)

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([40.169853, 26.641253, 52.715706, 23.691893, 49.462044],
      dtype=float32)>

In [211]:
loss

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [207]:
tf.divide(loss, tf.reduce_sum(tf.square(x_batch), axis=1))

InvalidArgumentError: {{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [10] vs. [5] [Op:RealDiv]

In [210]:
tf.square(x_batch)

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.7227235e+00, 7.8841519e-01, 3.3305879e+00, 2.5806174e+00,
        2.8236456e+00, 2.1111879e+01, 2.4590995e+00, 7.8797305e-01,
        2.4283993e+00, 1.1365116e+00],
       [3.9215775e+00, 1.5762302e+00, 6.6056151e+00, 1.1644359e-01,
        5.6876745e+00, 9.0521502e-01, 4.3535156e+00, 2.0891049e+00,
        1.0273392e+00, 3.5853818e-01],
       [9.7422391e-01, 1.2565459e-01, 3.8885303e-02, 9.2241801e-02,
        1.5771321e+01, 2.3519553e-01, 3.0575569e+00, 8.9186935e+00,
        1.3531401e+01, 9.9705334e+00],
       [8.3538862e-03, 5.7021980e+00, 1.4575380e+00, 7.8815589e+00,
        3.7042454e-02, 1.1443394e-03, 4.7315583e-01, 7.8459969e+00,
        1.4129360e-04, 2.8476232e-01],
       [5.3385362e-02, 2.4166159e-01, 2.2355444e+00, 6.5107121e+00,
        8.3945742e+00, 2.3514077e-02, 2.4994141e+01, 4.3666854e+00,
        2.2740734e+00, 3.6775467e-01]], dtype=float32)>

## PA-Classiers (binary classification)

![PA](images/PA_binary_classifiers.png)

In [183]:

def batch_train(initial_weights, batch, C):
    def t_pa(loss, x):
        return tf.divide(loss, tf.tensordot(x, x, axes=1))
    def t_pa1(x_batch, loss, C):
        return tf.minimum(C, tf.divide(loss, tf.reduce_sum(tf.square(x_batch), axis=1)))
    def t_pa2(loss, x, C):
        return tf.divide(loss, tf.add(tf.tensordot(x, x, axes=1), tf.divide(1, tf.multiply(2, C))))

    def _train_on_batch(weights, batch, C, t_fn):
        x_batch, y_batch = batch
        
        # predict y
        y_predict_batch = tf.sign(tf.tensordot(x_batch, weights, axes=1))
        # suffer loss
        loss = tf.maximum(0, tf.subtract(1, tf.multiply(y, tf.tensordot(x_batch, model, axes=1))))
        # update 1. Set
        t = t_fn(loss, x, C)
        # update 1. Update
        weights = tf.reduce_mean(tf.multiply(tf.expand_dims(tf.multiply(t, y_batch), axis=1), x_batch), axis=0)
        
        return weights
    
    return _train_on_batch(initial_weights, batch, C, t_pa1)    

In [132]:
@tf.function
def accuracy_test(weights, test_dataset):
    correct = 0
    total = 0
    
    for batch in iter(test_dataset):
        for x, y in zip(batch[0], batch[1]):
            y_predict = tf.sign(tf.tensordot(weights, x, axes=1))
            if y_predict.numpy() == y.numpy(): correct += 1
            total += 1
    return correct / total

In [184]:
#%%timeit -n 1 -r 1

model = tf.zeros(shape=(d,), dtype=tf.float32)

for epoch in range(1):
    for batch in iter(preprocessed_test_dataset):
        model = batch_train(model, batch, 0.01)

InvalidArgumentError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [100] vs. [10] [Op:Mul]

In [ ]:
accuracy_test(model, preprocessed_test_dataset)